In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

df = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv").withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))

df.show()

+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
|    user_id| device_id|referrer|                host|       url|          event_time|         event_date|
+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
| 1037710827| 532630305|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|
|  925588856| 532630305|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|
|-1180485268| 532630305|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|
|-1044833855| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|
|  747494706| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-26 16:03:...|2021-09-26 00:00:00|
|  747494706| 532630305|    NULL|admin.zachwilson....|         /|2021-02-21 16:08:...|2021-02-21 00:00:00|
| -824540328| 532630305|    NULL|admi

In [7]:
sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date"), col("host"), col("browser_family")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \

sorted.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `browser_family` cannot be resolved. Did you mean one of the following? [`user_id`, `referrer`, `device_id`, `event_date`, `event_time`].;
'Sort [event_date#175 ASC NULLS FIRST, host#166 ASC NULLS FIRST, 'browser_family ASC NULLS FIRST], false
+- RepartitionByExpression [event_date#175], 10
   +- Project [user_id#163, device_id#164, referrer#165, host#166, url#167, event_time#168, date_trunc(day, cast(event_time#168 as timestamp), Some(Etc/UTC)) AS event_date#175]
      +- Relation [user_id#163,device_id#164,referrer#165,host#166,url#167,event_time#168] csv


In [8]:
sorted2 = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("host"), col("event_date") ) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \

sorted2.show()

+----------+-----------+--------+--------------------+--------------------+--------------------+-------------------+
|   user_id|  device_id|referrer|                host|                 url|          event_time|         event_date|
+----------+-----------+--------+--------------------+--------------------+--------------------+-------------------+
|-106956778| -160197158|    NULL|aashish.techcreat...|              /login|2023-03-20 02:27:...|2023-03-20 00:00:00|
|1129583063|  532630305|    NULL|admin.zachwilson....|                   /|2021-01-07 09:21:...|2021-01-07 00:00:00|
|1272828233| -643696601|    NULL|admin.zachwilson....|                   /|2021-01-10 05:00:...|2021-01-10 00:00:00|
|1272828233| -643696601|    NULL|admin.zachwilson....|                   /|2021-01-10 10:32:...|2021-01-10 00:00:00|
|1272828233| -643696601|    NULL|admin.zachwilson....|                   /|2021-01-10 17:53:...|2021-01-10 00:00:00|
| 627059804|  532630305|    NULL|admin.zachwilson....|          

In [9]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [55]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [56]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));


++
||
++
++

In [112]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [113]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [120]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    

first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("browser_family"), col("host"))

sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [121]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files





size,num_files,sorted
2896920,4,sorted
3211534,4,unsorted


In [90]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
3145713,5


In [ ]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

count(1)
3665
